In [1]:
import os
from time import time
import yaml

from argparse import ArgumentParser, Namespace

from s3ts.api.nets.wrapper import WrapperModel
from s3ts.helper_functions import load_dm, str_time, get_parser

from pytorch_lightning import Trainer
from pytorch_lightning.utilities.model_summary import summarize

from s3ts.helper_functions import get_model

import torch
import numpy as np

import matplotlib.pyplot as plt

In [2]:
p = get_parser()

In [3]:
args = p.parse_args("--mode ts --dataset UCI-HAR --lr 0.001 --subjects_for_test 21 --window_size 25 --window_stride 1 --batch_size 128 --encoder_architecture cnn_gap --encoder_features 20 --decoder_architecture mlp --decoder_features 32 --decoder_layers 1 --pattern_size 32 --rho 0.1 --compute_n 300 --pattern_type syn --num_workers 8 --max_epochs 3 --normalize --reduce_imbalance --training_dir training_cv2 --n_val_subjects 4 --weight_decayL1 0 --weight_decayL2 0".split())

In [4]:
dm = load_dm(args)

class_changes = [0] + list(np.nonzero(np.diff(dm.stsds.SCS))[0])
print(len(class_changes), "number of class changes")

Loaded dataset UCI-HAR with a total of 815614 observations for window size 25
Sampling 59084 (balanced) observations per epoch.
Using 708997 observations for training, 106617 for validation and 25512 observations for test
1787 number of class changes


In [52]:
def process_fft_full(STS): 
    # no tiene sentido hacer esto, ya que frecuencias que van a tener mayor peso son relacionadas con muchos sujetos y muchas clases a la vez
    # como la serie temporal es larga no funciona
    
    fft_size = 2**int(np.log2(STS.shape[1]))
    fft_short = np.fft.fft(STS, axis=-1, n=fft_size)
    fft_freq = np.fft.fftfreq(fft_size, d=1/50)

    return fft_short, fft_freq

In [53]:
out = process_fft_full(dm.stsds.STS)

In [54]:
sorted = np.argsort(np.abs(out[0]), axis=-1)
freq_sorted = out[1][sorted]

In [55]:
freq_sorted.shape

(6, 1048576)

In [49]:
1/1048576

9.5367431640625e-07

In [60]:
freq_sorted[:,-20:]

array([[ 4.29153442e-04, -4.29153442e-04, -1.62601471e-02,
         1.62601471e-02, -4.95910645e-03,  4.95910645e-03,
         2.86102295e-03, -2.86102295e-03, -1.71184540e-02,
         1.71184540e-02,  2.95639038e-03, -2.95639038e-03,
         2.38418579e-03, -2.38418579e-03,  3.00407410e-03,
        -3.00407410e-03,  2.57492065e-03, -2.57492065e-03,
         2.52723694e-03, -2.52723694e-03],
       [-2.71797180e-03,  2.71797180e-03, -4.95910645e-03,
         4.95910645e-03,  1.43051147e-04, -1.43051147e-04,
        -2.81333923e-03,  2.81333923e-03, -2.86102295e-03,
         2.86102295e-03, -2.67028809e-03,  2.67028809e-03,
         2.95639038e-03, -2.95639038e-03, -3.00407410e-03,
         3.00407410e-03, -2.57492065e-03,  2.57492065e-03,
        -2.52723694e-03,  2.52723694e-03],
       [-4.72068787e-03,  4.72068787e-03,  3.24249268e-03,
        -3.24249268e-03,  9.05990601e-04, -9.05990601e-04,
        -2.86102295e-03,  2.86102295e-03, -1.95503235e-03,
         1.95503235e-03, -6.6